In [1]:
import json
import pandas as pd

In [2]:
# For some reason, loading into a dictionary first and then converting to a df
# is faster than just directly loading into a df. Strange.
f = open('./data/all_movies_data_new.json', 'r')
movies_data = json.load(f)
print("loaded")

df = pd.DataFrame.from_dict(movies_data)

loaded


In [3]:
# The translation from the raw data to DataFrame form is transposed the opposite way we want it.
# Let's fix that
df = df.T

In [4]:
# Let's see how many entries we have
print(f'There are {len(df)} movies!')

There are 275231 movies!


In [5]:
# This looks much better!
display(df.head())

# But now we have some entries that unfortunately didn't get pulled correctly from the API
# These all have status code of 34. Let's remove these rows.
df = df[df['status_code'] != 34]

# There were some slight issues in the detection of titles initially.
# Let's only keep movies that are originally in English
df = df[df['original_language'] == 'en']

# Also, this database has movies that may have only been planned but never released.
# Let's remove any that haven't been released
print(df.status.value_counts())
df = df[df['status'] == 'Released']

print(f'There are {len(df)} movies after dropping issues!')

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,success,status_code,status_message
3,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",,3,tt0092149,fi,Varjoja paratiisissa,...,"[{'english_name': 'Swedish', 'iso_639_1': 'sv'...",Released,,Shadows in Paradise,False,7.263,211,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,34,The resource you requested could not be found.
8,False,None,None,42000,"[{'id': 99, 'name': 'Documentary'}]",http://lifeinloops.com,8,tt0825671,en,Life in Loops (A Megacities RMX),...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.7,21,NaN,NaN,NaN
9,False,None,None,0,"[{'id': 18, 'name': 'Drama'}]",,9,tt0425473,de,Sonntag im August,...,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,,Sunday in August,False,7.4,14,NaN,NaN,NaN
12,False,/yzqaKAhglTrkeOfuIXYYArf0WnA.jpg,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",http://movies.disney.com/finding-nemo,12,tt0266543,en,Finding Nemo,...,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,There are 3.7 trillion fish in the ocean. They...,Finding Nemo,False,7.823,17305,NaN,NaN,NaN


Released           114510
Planned               412
In Production         182
Post Production       142
Canceled               36
Rumored                35
Name: status, dtype: int64
There are 114510 movies after dropping issues!


In [6]:
# Show the columns
print(df.columns)
# Drop these unnecessary columns
df = df.drop(columns=['adult', 'backdrop_path', 'belongs_to_collection', 'homepage', 'production_companies', 'production_countries', 'video',
'original_language', 'success', 'status_code', 'status_message', 'status'])

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'success', 'status_code',
       'status_message'],
      dtype='object')


In [7]:
# Our genres are in dictionary form. These will be cumbersome, hard to work with, and take up more memory than needed.
# Let's convert them to lists
genres = df['genres']
genres = list(genres)

list_genres = []
for entry in genres:
    temp_list = []
    for genre in entry:
        temp_list.append(genre['name'])
    list_genres.append(temp_list)

df['genres'] = list_genres

In [8]:
# We can do the same thing for spoken_languages
langs = df['spoken_languages']
langs = list(langs)

list_langs = []
for entry in langs:
    temp_list = []
    for lang in entry:
        temp_list.append(lang['english_name'])
    list_langs.append(temp_list)

df['spoken_languages'] = list_langs

In [9]:
# Great! Now we're done cleaning and we can save this dataframe as a csv
df.to_csv('./data/initial_cleaned_data.csv')